### Install Required Libraries

In [1]:
!pip install folium torch scikit-learn matplotlib streamlit joblib

### Import Libraries

In [2]:
import pandas as pd
import numpy as np
import requests
import folium
from folium.plugins import HeatMap
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import joblib
import time
import streamlit.components.v1 as components

### Fetch Earthquake Data from USGS

In [3]:
def get_usgs_quake_data(start_time="2024-12-01", end_time="2024-12-31", min_magnitude=0, max_magnitude=2):
    url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
    params = {
        "format": "geojson",
        "starttime": start_time,
        "endtime": end_time,
        "minmagnitude": min_magnitude,
        "maxmagnitude": max_magnitude,
    }

    response = requests.get(url, params=params)
    data = response.json()
    
    records = []
    for feature in data["features"]:
        props = feature["properties"]
        coords = feature["geometry"]["coordinates"]
        records.append({
            "place": props["place"],
            "magnitude": props["mag"],
            "time": pd.to_datetime(props["time"], unit="ms"),
            "longitude": coords[0],
            "latitude": coords[1],
            "depth": coords[2],
        })
        
    return pd.DataFrame(records)

df_quakes = get_usgs_quake_data()
df_quakes.head()

,place,magnitude,time,longitude,latitude,depth
0,"60 km S of Kingston, Nevada",1.70,2024-12-30 23:48:43.692,-117.093600,38.662100,8.30
1,"3 km ENE of The Geysers, CA",1.04,2024-12-30 23:48:41.250,-122.733665,38.792000,1.76
2,"51 km SSE of Nelchina, Alaska",1.50,2024-12-30 23:39:37.619,-146.250200,61.605100,24.10
3,"57 km WSW of Adak, Alaska",1.80,2024-12-30 23:37:54.324,-177.329000,51.589300,18.80
4,"6 km WNW of Cobb, CA",0.34,2024-12-30 23:27:31.700,-122.788667,38.823167,1.61


### Geospatial Mapping

In [4]:
def plot_quakes_on_map(df):
    m = folium.Map(location=[df.latitude.mean(), df.longitude.mean()], zoom_start=5)
    for _, row in df.iterrows():
        folium.CircleMarker(
            location=[row["latitude"], row["longitude"]],
            radius=row["magnitude"] * 2,
            popup=f"{row['place']} | Mag: {row['magnitude']:.1f}",
            color="red" if row["magnitude"] > 1.5 else "orange",
            fill=True,
            fill_opacity=0.6
        ).add_to(m)
    return m

quake_map = plot_quakes_on_map(df_quakes)
quake_map.save("quake_map.html")
components.html(open("quake_map.html").read(), height=500)

2025-04-06 01:40:21.406 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 01:40:21.486 
  command:

    streamlit run /Users/syukrifadzil/Library/Python/3.13/lib/python/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-04-06 01:40:21.487 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

### Seismic Heatmap

In [5]:
def plot_quake_heatmap(df):
    m = folium.Map(location=[df.latitude.mean(), df.longitude.mean()], zoom_start=5)
    heat_data = [[row['latitude'], row['longitude'], row['magnitude']] for index, row in df.iterrows()]
    HeatMap(heat_data).add_to(m)
    return m

heat_map = plot_quake_heatmap(df_quakes)
heat_map.save("heat_map.html")
components.html(open("heat_map.html").read(), height=500)

2025-04-06 02:04:15.852 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 02:04:15.855 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

### Simulate Seismic Data Sensor

In [6]:
df = pd.DataFrame({
    'accel_x': np.random.rand(1000),
    'accel_y': np.random.rand(1000),
    'accel_z': np.random.rand(1000),
    'precursor_label': np.random.choice([0, 1], size=1000)
})

features = ['accel_x', 'accel_y', 'accel_z']
X = df[features].values
y = df['precursor_label'].values

### Train Classic ML Model

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RandomForestClassifier(n_estimators=100)
rf_model.fit(X_train, y_train)

svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

logreg_model = LogisticRegression()
logreg_model.fit(X_train, y_train)

print("Random Forest Accuracy:", rf_model.score(X_test, y_test))
print("SVM Accuracy:", svm_model.score(X_test, y_test))
print("Logistic Regression Accuracy:", logreg_model.score(X_test, y_test))

Random Forest Accuracy: 0.455
SVM Accuracy: 0.49
Logistic Regression Accuracy: 0.495


### Prepare Data for LSTM (PyTorch)

In [8]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

def create_sequence(data, labels, seq_length=50):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x_seq = data[i:i+seq_length]
        y_seq = labels[i+seq_length]
        xs.append(x_seq)
        ys.append(y_seq)
    return np.array(xs), np.array(ys)

X_seq, y_seq = create_sequence(X_scaled, y)
X_seq = torch.tensor(X_seq, dtype=torch.float32)
y_seq = torch.tensor(y_seq, dtype=torch.float32)

### Define and Train LSTM with PyTorch

In [9]:
class EarthquakeLSTM(nn.Module):
    def __init__(self, input_size=3, hidden_size=64, num_layers=1):
        super(EarthquakeLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return self.sigmoid(out)
    
model = EarthquakeLSTM()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    outputs = model(X_seq)
    loss = criterion(outputs.squeeze(), y_seq)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():4f}")

Epoch 1/10, Loss: 0.693108
Epoch 2/10, Loss: 0.693303
Epoch 3/10, Loss: 0.693149
Epoch 4/10, Loss: 0.693502
Epoch 5/10, Loss: 0.693201
Epoch 6/10, Loss: 0.692773
Epoch 7/10, Loss: 0.693725
Epoch 8/10, Loss: 0.693233
Epoch 9/10, Loss: 0.693158
Epoch 10/10, Loss: 0.692843


### Save Models

In [10]:
torch.save(model.state_dict(), "lstm_quake_detector.pt")
joblib.dump(rf_model, "rf_quake.pkl")
joblib.dump(svm_model, "svm_quake.pkl")
joblib.dump(logreg_model, "logreg_quake.pkl")
joblib.dump(scaler, "scaler.pkl")


['scaler.pkl']

### Real-Time Prediction

In [13]:
def generate_mock_data():
    return {
        "accel_x": np.random.rand(),
        "accel_y": np.random.rand(),
        "accel_z": np.random.rand(),
    }
    
def predict_all_models(sensor_window, model, scaler):
    sensor_scaled = scaler.transform(sensor_window)
    tensor_input = torch.tensor([sensor_scaled], dtype=torch.float32)
    
    with torch.no_grad():
        lstm_pred = model(tensor_input).item()
    
    # Classic models use only the most recent reading (last row of the window)
    latest_reading = np.array(sensor_window[-1]).reshape(1, -1)
    latest_scaled = scaler.transform(latest_reading)
    
    rf_pred = rf_model.predict(latest_scaled)[0]
    svm_pred = svm_model.predict(latest_scaled)[0]
    logreg_pred = logreg_model.predict(latest_scaled)[0]
    
    return {
        'LSTM': lstm_pred > 0.5,
        'Random Forest': bool(rf_pred),
        'SVM': bool(svm_pred),
        'Logistic Regression': bool(logreg_pred)
    }
    
# Simulate 50 real-time readings
buffer = []
for _ in range(50):
    mock = generate_mock_data()
    buffer.append([mock["accel_x"],mock["accel_y"],mock["accel_z"]])
    if len(buffer) >= 50:
        result = predict_all_models(buffer[-50:], model, scaler)
        print("Prediction:", result)
        time.sleep(1)

Prediction: {'LSTM': False, 'Random Forest': True, 'SVM': False, 'Logistic Regression': False}
